In [ ]:
# @markdown ## 動くサンプル ～ 活用事例 / 水域・森林などの種類ごとに、土地の構成割合を算出
# @markdown この事例のAIを用いた、衛星画像データから土地の種類を分類する分析（水域・森林など）、をお試しできます。下記の手順でご利用ください。
# @markdown 1. 左にある▶ボタン（再生ボタンのような）を押してください。
# @markdown     * `警告: このノートブックは Google が作成したものではありません。`と警告表示がされますが、`このまま続行`としてください。
# @markdown 2. `👉ユーザー名を入力してください: ` `👉パスワードを入力してください: ` と表示されたら、ショウケースサイトの認証情報を入力してください。
# @markdown 3. しばらく待つと、動くサンプルの画面が表示されます。`分析実行`ボタンから、分析を実行できます。
# @markdown     * 任意) 対象の日時を変更する場合は、`対象期間（年月）`欄に入力してください。
# @markdown     * 任意) 対象の領域を変更する場合は、`矩形選択`ボタンを押し、マウスのドラッグ＆ドロップで領域を選択してください。
# @markdown **既知の問題)** 分析結果が表示されなくなる場合があります。その際は、再度▶ボタンを押してやり直してください。

import ipywidgets
import IPython.display
from pathlib import Path

# おまじない
from google.colab import output
output.enable_custom_widget_manager()

ZIP_URL = "https://main.d4dnk2dwq768l.amplifyapp.com/files/usecases/2/demo-app.zip"
ZIP_FILE = "demo-app.zip"
WORK_DIR = Path('demo-app')

def download_and_unzip() -> None:
    !echo "必要なファイルをダウンロードします... $ZIP_URL" && echo && \
        read -p "👉ユーザー名を入力してください: " __user && \
        read -sp "👉パスワードを入力してください: " __pass && echo && \
        curl --progress-bar \
            -fL $ZIP_URL \
            -o $ZIP_FILE \
            --basic -K- <<< "-u $$__user:$$__pass" && \
        echo "ダウンロードしました." || echo "エラーが発生しました. ユーザー名/パスワードを確認してください."

    zip_file = Path(ZIP_FILE)
    if not zip_file.exists() or zip_file.stat().st_size <= 0:
        raise

    !unzip -o $ZIP_FILE > /dev/null
        # 標準出力は非表示（エラー出力のみ表示）

    if not WORK_DIR.exists():
        raise

def pip_install() -> None:
    print("ライブラリをインストールします...")
    requirements_file = str(WORK_DIR / "requirements.txt")
    !pip install -r $requirements_file > /dev/null
        # 標準出力は非表示（エラー出力のみ表示）
        # -qでも良いが（INFOレベルは出力しない）、ダウンロード進捗バーなどは出る
    print("ライブラリをインストールしました.")

# https://github.com/jupyter-widgets/ipywidgets/issues/3208
_output = ipywidgets.Output(
    layout=ipywidgets.Layout(
        width='100%',
    ),
)
display(_output)

setuped = setuped if "setuped" in locals() else False
with _output:
    if not setuped:
        IPython.display.clear_output()
        print("セットアップ開始...")
        download_and_unzip()
        pip_install()
        print("セットアップ終了.")
        setuped = True

import sys
sys.path.append(str(WORK_DIR.resolve()))

import importlib
importlib.invalidate_caches()
import app
importlib.reload(app)
from app import App

_output.clear_output()
if "_app" in locals():
    _app_new = App(
        map_center=_app.map_center,
        map_zoom=_app.map_zoom,
        roi=_app.roi,
        yyyymm=_app.yyyymm,
    )
    _app = _app_new
else:
    _app = App()

_app.show()